In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader


In [2]:
import numpy as np

# 设置随机种子以确保可重复性
np.random.seed(42)
torch.manual_seed(42)

# 定义常量
num_nodes = 100  # 节点数量
timesteps = 10   # 时间步数
feature_dim = 4  # 节点特征维度（采纳率、产量、天气、土壤肥力）
ext_feature_dim = 2  # 外部特征维度（市场和政策）

# 1. 生成节点特征
adoption_rate = np.random.rand(num_nodes, 1, timesteps)   # 采纳率
yield_data = np.random.rand(num_nodes, 1, timesteps)      # 产量
weather_data = np.random.rand(num_nodes, 1, timesteps)     # 天气
soil_fertility = np.random.rand(num_nodes, 1, timesteps)   # 土壤肥力

# 合并特征，形成形状为 [num_nodes, feature_dim, timesteps]
x = np.concatenate((adoption_rate, yield_data, weather_data, soil_fertility), axis=1)  # shape: [num_nodes, feature_dim, timesteps]

# 2. 生成边特征（地理位置和社交网络）
# 示例：随机生成边（地理位置和社交网络关系）
num_edges = 200  # 定义边的数量
edge_index = np.random.randint(0, num_nodes, (2, num_edges))  # 形状：[2, num_edges]

# 3. 生成外部特征（市场、政策）
market_data = np.random.rand(num_nodes, ext_feature_dim, timesteps)  # 形状：[num_nodes, ext_feature_dim, timesteps]

# 4. 生成目标变量（这里是一个示例）
y = np.random.rand(num_nodes)  # 目标值（例如：未来的产量）

# 5. 创建 Data 对象
x_tensor = torch.tensor(x, dtype=torch.float32)  # 转换为 Tensor
edge_index_tensor = torch.tensor(edge_index, dtype=torch.long)  # 转换为 Tensor
y_tensor = torch.tensor(y, dtype=torch.float32)  # 转换为 Tensor
market_data_tensor = torch.tensor(market_data, dtype=torch.float32)  # 转换为 Tensor

data = Data(x=x_tensor, edge_index=edge_index_tensor, y=y_tensor, ext_features=market_data_tensor)

# 6. 创建数据加载器
dataset = [data] * 100  # 生成 100 个样本
train_loader = DataLoader(dataset[:80], batch_size=16, shuffle=True)  # 训练集
test_loader = DataLoader(dataset[80:], batch_size=16)  # 测试集

# 输出样本数据
print("Sample Node Features (x):", x_tensor[0])
print("Sample Edge Index:", edge_index_tensor)
print("Sample External Features:", market_data_tensor[0])
print("Sample Target (y):", y_tensor[0])


Sample Node Features (x): tensor([[0.3745, 0.9507, 0.7320, 0.5987, 0.1560, 0.1560, 0.0581, 0.8662, 0.6011,
         0.7081],
        [0.1851, 0.5419, 0.8729, 0.7322, 0.8066, 0.6588, 0.6923, 0.8492, 0.2497,
         0.4894],
        [0.2617, 0.2470, 0.9063, 0.2495, 0.2719, 0.7594, 0.4497, 0.7767, 0.0654,
         0.4876],
        [0.6727, 0.7967, 0.2505, 0.6249, 0.5717, 0.8328, 0.9061, 0.0122, 0.6740,
         0.0518]])
Sample Edge Index: tensor([[20, 63, 36, 26, 54, 28, 19, 24, 69, 62, 35,  1, 21, 95, 15, 17, 68, 53,
         99, 29, 44, 18, 86, 33, 12, 42, 61, 92, 99, 21, 51, 51, 35, 72, 18, 37,
         69, 78, 56, 19,  8, 31,  3, 49, 81,  4, 19, 86, 74, 36, 21, 57, 67, 92,
         43, 33, 76, 41,  8, 94,  6, 67, 18, 93, 30, 34, 56, 84, 60, 30, 82, 94,
         52, 31, 65, 58,  9,  1, 65, 59, 87, 16, 95, 15, 15, 45, 38, 30, 87,  0,
         10, 84,  3, 70, 25, 49, 11, 94, 80, 96, 74, 91, 95, 64, 46, 91, 52, 70,
          1, 23,  3, 18, 25, 74, 39, 49, 42, 74, 66, 11, 66, 26, 53, 65,

C:\Users\34184\anaconda3\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [3]:
class TemporalConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3):
        super(TemporalConvLayer, self).__init__()
        # 使用padding确保输出的时间步长与输入一致
        self.conv1d = nn.Conv1d(in_channels, out_channels, kernel_size, padding=(kernel_size // 2))

    def forward(self, x):
        # x 的形状: [batch, features, timesteps]
        x = self.conv1d(x)
        return torch.relu(x)


In [4]:
class SpatialAttentionLayer(nn.Module):
    def __init__(self, in_channels):
        super(SpatialAttentionLayer, self).__init__()
        self.attention_weights = nn.Linear(in_channels, 1)

    def forward(self, x, edge_index):
        # x 的形状: [num_nodes, in_channels]
        attention_scores = torch.sigmoid(self.attention_weights(x))
        x = x * attention_scores
        return x


In [5]:
class STGCNWithAttention(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, ext_feature_dim):
        super(STGCNWithAttention, self).__init__()
        self.temporal1 = TemporalConvLayer(in_channels + ext_feature_dim, hidden_channels)
        self.spatial = SpatialAttentionLayer(hidden_channels)
        self.temporal2 = TemporalConvLayer(hidden_channels, out_channels)
        self.fc = nn.Linear(out_channels, 1)

    def forward(self, x, edge_index, ext_features):
        batch_size, num_nodes, feat_dim, timesteps = x.shape
        ext_features = ext_features.unsqueeze(1).repeat(1, num_nodes, 1, timesteps)
        x = torch.cat([x, ext_features], dim=2)

        # 第一层时间卷积
        x = self.temporal1(x)
        # 进行空间卷积
        x = x.view(-1, x.size(2), x.size(3))
        x = self.spatial(x, edge_index)
        x = x.view(batch_size, num_nodes, -1, x.size(-1))

        # 第二层时间卷积
        x = self.temporal2(x)
        x = x.view(batch_size, -1)
        x = self.fc(x)
        return x


In [6]:
num_nodes = 100
timesteps = 10
feature_dim = 5
ext_feature_dim = 3

x = torch.rand((num_nodes, feature_dim, timesteps))
edge_index = torch.tensor([[0, 1, 2], [1, 2, 3]], dtype=torch.long)
ext_features = torch.rand((ext_feature_dim, timesteps))
y = torch.rand((num_nodes,))

data = Data(x=x, edge_index=edge_index, y=y, ext_features=ext_features)

dataset = [data] * 100
train_loader = DataLoader(dataset[:80], batch_size=16, shuffle=True)
test_loader = DataLoader(dataset[80:], batch_size=16)


In [7]:
def train(model, data_loader, optimizer, criterion, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for data in data_loader:
            optimizer.zero_grad()
            x, edge_index, y = data.x, data.edge_index, data.y
            output = model(x, edge_index, data.ext_features)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(data_loader):.4f}")


In [8]:
model = STGCNWithAttention(in_channels=5, hidden_channels=32, out_channels=16, ext_feature_dim=3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

train(model, train_loader, optimizer, criterion, epochs=10)


ValueError: not enough values to unpack (expected 4, got 3)